# Actividad integradora
## Robot organizador de almacen

### Equipo 2:
- Carlos Tadeo Pérez Capistrán, A01197315
- David Fernando Armendáriz Torres, A01570813
- Nicolás Herrera Hernandez, A01114972




## Imports
 

In [1]:
#!pip install mesa

In [1]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid, MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo y los agentes


In [43]:
# Agente de la aspiradora o robot limpiador
class Robot(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos, el numero de cajas que carga el robot y el valor de color, este nos va a servir para graficar.
    self.color = 6
    self.moves = 0
    self.boxes = 0

# Función del agente para dejar la pila de cajas en el piso
  def placeStack(self):
    for cell in self.model.grid.coord_iter():
      cell_content, x, y = cell
      for content in cell_content:
        if isinstance(content, Caja):
          if content.pos == self.pos:
              if content.color == 0:
                content.color = self.boxes
                self.boxes = 0

# Función del agente para agarrar la caja que se encuentra en la misma celda que el
# y reducir el numero de cajas en el modelo
  def getBox(self):
    for cell in self.model.grid.coord_iter():
      cell_content, x, y = cell
      for content in cell_content:
          if isinstance(content, Caja):
            if content.pos == self.pos:
              if content.color == 1:
                content.color = 0
                self.boxes += 1
                self.model.boxCount -= 1

  def step(self):
    # Los vecinos de nuestro agente
    neighbours = self.model.grid.get_neighbors(
      self.pos,
      moore=False,
      include_center=False)
    collisionPositions = [[]]
    # Para cada vecino de los vecinos
    for neighbour in neighbours:
      # Si nuestro vecino tiene un color mayor a 1, es decir,
      # es un robot o una pila de cajas se agrega la posicion a las
      # posiciones de colision
      if neighbour.color > 1:
        collisionPositions.append(neighbour.pos)
    if self.boxes == 5 or self.boxes > self.model.boxCount:
      self.placeStack()
    else:
      possible_steps = self.model.grid.get_neighborhood(
        self.pos,
        moore=False,
        include_center=False)
      # Se eliminan las posiciones donde hay colision de las posibles posiciones nuevas. 
      for pStep in possible_steps:
        for cPos in collisionPositions:
          if cPos == pStep and possible_steps.count(pStep) > 0:
            possible_steps.remove(pStep)
      # En caso de que el agente no se pueda mover se queda en la misma posición 
      if len(possible_steps) == 0:
        self.model.grid.move_agent(self, self.pos)
      else:
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        self.moves += 1
    self.getBox()

# Agente de la Caja
class Caja(Agent):
  def __init__(self, unique_id, model, status):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, donde 1 = caja y 0 = no hay caja.
    self.color = status
    self.moves = 0
    

class Modelo(Model):
  def __init__(self, N, width, height, K):
    self.num_agents = N
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.running = True
    self.boxes = K
    self.boxCount = self.boxes
    self.dirty_list = [[1,1]]
    
    # Ubicamos a cada los robots en la grilla de acuerdo al numero de agentes indicado
    for i in range(self.num_agents):
      a = Robot(i, self)
      self.schedule.add(a)
      self.grid.place_agent(a, (1, 1))

    b = Caja((1, 1), self, 0)
    self.schedule.add(b)
    self.grid.place_agent(b, (1,1))
    
    # Ubicamos a cada Caja en la grilla
    cont = 0
    while cont != self.boxes:
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      b = Caja((x, y), self, 1)
      if ([x,y] not in self.dirty_list):
        self.schedule.add(b)
        self.grid.place_agent(b, (x,y))
        cont = cont + 1
      self.dirty_list.append([x,y])  
    
    # Ubicamos al piso en la grilla
    for (content, x, y) in self.grid.coord_iter():
      if ([x,y] not in self.dirty_list):
        b = Caja((x, y), self, 0)
        self.schedule.add(b)
        self.grid.place_agent(b, (x, y))

    # Aquí definimos el colector de datos para obtener el grid completo y los parametros de los agentes.
    self.datacollector = DataCollector(
    model_reporters={"Grid": self.get_grid}, agent_reporters={"Color": "color", "Moves": "moves"}
    )

  def step(self):
    self.schedule.step()
    self.datacollector.collect(self)


  def get_grid(self):

      # Generamos la grid para contener los valores
      grid = np.zeros((self.grid.width, self.grid.height))

      # Asignamos una celda a cada uno de los elementos de la grilla
      for cell in self.grid.coord_iter():
          cell_content, x, y = cell
          for agent in cell_content:
            grid[x][y] = agent.color

      return grid


# Ejecución del modelo
A continuación corremos el modelo con los valores proporcionados.

In [83]:
# Definimos el tamaño del Grid, procentaje de celdas sucias y num de agentes
GRID_WIDTH = 7
GRID_HEIGHT = 7
BOXES = 12
NUM_AGENTS = 2

# Definimos el tiempo limite de ejecucion
MAX_EXEC_TIME = 3.5

# Creamos el modelo con las valores anteriores
empty_model = Modelo(NUM_AGENTS, GRID_WIDTH, GRID_HEIGHT, BOXES)

# Registramos el tiempo de inicio
start_time = time.time()
total_time = str(datetime.timedelta(seconds=MAX_EXEC_TIME))

# Corremos el modelo mientras que no se cumpla el tiempo limite y que todas las cajas no esten apiladas
while((time.time() - start_time) < MAX_EXEC_TIME and not empty_model.boxCount == 0):
  empty_model.step()

# Guardamos el tiempo que le tomó correr al modelo.
exec_time =  str(datetime.timedelta(seconds=(time.time() - start_time)))


# Datos a analizar
Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información. Con esta información podremos calcular lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

In [71]:
# Obtenemos la información recolectada del modelo
all_grid = empty_model.datacollector.get_model_vars_dataframe()

# Obtenemos la información recolectada de los agentes
agent_vars = empty_model.datacollector.get_agent_vars_dataframe().reset_index()

# Seleccionamos los datos de movimiento de los agentes en el ultimo paso del modelo
all_moves = agent_vars[agent_vars['Step'] == agent_vars['Step'].values[-1]].dropna()['Moves']
last_step_values = agent_vars[agent_vars['Step'] == agent_vars['Step'].values[-1]].dropna()

print('Movimientos realizados por los agentes: ', all_moves.sum())
print('Tiempo de necesario para que todas las cajas apiladas: ', exec_time, '/', total_time)

Movimientos realizados por los agentes:  82
Tiempo de necesario para que todas las cajas apiladas:  0:00:00.004985 / 0:00:03.500000


# Graficación
Graficamos la información usando `matplotlib`

In [72]:
%%capture
# Obtenemos el numero de pasos del recolector de datos
steps = len(all_grid)
cmap = matplotlib.cm.get_cmap('viridis', 7) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([256/256, 256/256, 256/256, 1])   # piso blanco
cmap[1] = np.array([213/256, 184/256, 149/256, 1])   # caja beige
cmap[2] = np.array([194/256, 155/256, 106/256, 1])   # stack beige claro
cmap[3] = np.array([168/256, 123/256, 68/256, 1])    # stack beige medio claro
cmap[4] = np.array([108/256, 81/256, 44/256, 1])     # stack beige oscuro
cmap[5] = np.array([52/256, 38/256, 21/256, 1])      # stack beige muy oscuro
cmap[6] = np.array([50/256, 50/256, 50/256, 1])      # robot gris
new_cmap = matplotlib.colors.ListedColormap(cmap)

fig, axs = plt.subplots(figsize=(GRID_HEIGHT, GRID_WIDTH))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])


anim = animation.FuncAnimation(fig, animate, frames=steps)

# Animación
Colores de celda:
- Gris: Robot.
- Blanco: Piso.
- Beige: Caja.
- Beige / Cafe : Pila de cajas entre más oscuro más cajas. 

In [73]:
anim

# Datos a analizar

In [74]:
# Movimientos realizados por los agentes
all_moves.head(NUM_AGENTS)

1134    41
1135    41
Name: Moves, dtype: int64

# Estrategia
Al realizar un análisis de distintas situaciones, incremento de agentes robot y de cajas, se pudieron observar distintos comportamientos. Nos pudimos dar cuenta que al incrementar los agentes se observaba una mejora de tiempo inicialmente, pero se llega un punto donde el número de agentes robot no ofrece una mejoría en cuanto a tiempo, esto se debe a que los agentes eligen su nueva posición de manera aleatoria además al incrementar los agentes se ve un aumento de los movimientos totales y disminución de movimientos individuales. 

El primer paso para implementar una estrategia que disminuya el número de movimientos y tiempo que le toman a los robots seria la implementación de un sistema inteligente de movimiento donde el robot sabe de donde viene y que celdas no cuentan con cajas de esta manera no pasa por las celdas que ya reviso y se disminuyen el número de movimientos al mismo tiempo reduciendo el tiempo total que le toma al modelo para ejecutarse. 

El segundo paso seria implementar unos estantes donde los robots pueden dejas las cajas que llevan en pilas, por el momento solo las dejan en el piso al llegar a cierta cantidad de cajas, esto es incorrecto no solo porque es ilógico, pero también porque están pilas van a funcionar como obstáculos y les van a quitar mas tiempo a los robots para recoger las otras cajas. Esto tambien evitaria que los robots queden atrapados entre las pilas de cajas como sucedia en algunos casos.
